# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
from pprint import pprint 

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_csv = "../output_data/cities.csv"

# Read the mouse data and the study results
cities_data = pd.read_csv(cities_csv)
cities_data

,city_ID,city,cloudiness,country,date,humidity,latitude,longitude,max_temp,wind_speed
0,0,bredasdorp,2.0,ZA,04/12/2020,62.0,-34.53,20.04,59.00,6.93
1,1,rikitea,62.0,PF,04/12/2020,85.0,-23.12,-134.97,78.15,15.39
2,2,east london,99.0,ZA,04/12/2020,64.0,-33.02,27.91,69.01,22.06
3,3,busselton,63.0,AU,04/12/2020,66.0,-33.65,115.33,62.01,8.75
4,4,leningradskiy,99.0,RU,04/12/2020,87.0,69.38,178.42,17.67,2.80
...,...,...,...,...,...,...,...,...,...,...
606,606,tessalit,38.0,ML,04/12/2020,14.0,20.20,1.01,89.96,6.17
607,607,vezirkopru,18.0,TR,04/12/2020,56.0,41.14,35.45,41.00,3.36
608,608,koutiala,67.0,ML,04/12/2020,24.0,12.39,-5.46,94.93,3.80
609,609,christchurch,5.0,NZ,04/12/2020,36.0,-43.53,172.63,61.00,18.34


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#configure gmaps
gmaps.configure(api_key=g_key)

In [4]:
#drop data w null values
complete_cities_data = cities_data.dropna()
#turn lat/lng into locations
locations = complete_cities_data[['latitude', 'longitude']].astype(float)
#convert humidity to float
humidity_rate = complete_cities_data['humidity'].astype(float)

In [5]:
# Create a Humidity Heatmap layer
fig = gmaps.figure(layout={'width': '650px', 'height': '500px'}, zoom_level=1.35, center=(41.886987, 12.498911))

#configure data for layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity_rate, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1, )
#add layer
fig.add_layer(heat_layer)

#display
fig



Figure(layout=FigureLayout(height='500px', width='650px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
#narrowing results down to ideal weather
ideal_temp_low_df = complete_cities_data[complete_cities_data.max_temp >= 62]
ideal_temp_high_low_df = ideal_temp_low_df[ideal_temp_low_df.max_temp < 69]
ideal_wind_df = ideal_temp_high_low_df[ideal_temp_high_low_df.wind_speed < 5]
ideal_humidity = ideal_wind_df[ideal_wind_df.humidity < 75]
ideal_cloud_cover = ideal_humidity[ideal_humidity.cloudiness > 25]
ideal_vacation_spot = pd.DataFrame(ideal_cloud_cover)
ideal_vacation_spot

,city_ID,city,cloudiness,country,date,humidity,latitude,longitude,max_temp,wind_speed
25,25,camabatela,100.0,AO,04/12/2020,70.0,-8.19,15.38,65.12,3.04
238,238,lyantonde,100.0,UG,04/12/2020,63.0,-0.40,31.16,65.35,3.11
284,284,kibondo,100.0,TZ,04/12/2020,66.0,-3.59,30.72,62.46,1.88
308,308,zvishavane,77.0,ZW,04/12/2020,35.0,-20.33,30.07,67.37,2.66
433,433,along,97.0,IN,04/12/2020,54.0,28.17,94.77,63.97,4.14
484,484,pittsburg,90.0,US,04/12/2020,67.0,38.03,-121.88,64.99,3.80
544,544,katete,100.0,UG,04/12/2020,61.0,-0.78,29.75,65.39,3.31
548,548,zambezi,60.0,ZM,04/12/2020,62.0,-13.54,23.10,66.27,3.89
579,579,chiradzulu,30.0,MW,04/12/2020,70.0,-15.67,35.14,62.82,2.95


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
ideal_vacation_spot['hotel name'] = ""


In [19]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
target_radius = 5000
params = {
    "type": "hotel",
    "radius": target_radius,
    "key": g_key,
    "keyword": "hotel"
}
# use iterrows to iterate through pandas dataframe
for index, row in ideal_vacation_spot.iterrows():

    # get restaurant type from df
    hotel_finder = str(row.latitude)+","+str(row.longitude)
    #print(hotel_finder)
    
    # add keyword to params dict
    params["location"] = hotel_finder
    
    #print(params)
    
    # assemble url and make API request
    ###print(f"Retrieving Results for Index {index}: {hotel_finder}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    pprint(results)
    
    try:
        print( f"The closest hotel is {results[0]['name']}.")
        
        ideal_vacation_spot.loc[index, 'hotel name'] = results[0]['name']
        ideal_vacation_spot.loc[index, 'hotel rating'] = results[0]['rating']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

[{'geometry': {'location': {'lat': -8.1922915, 'lng': 15.3712592},
               'viewport': {'northeast': {'lat': -8.191248870107279,
                                          'lng': 15.37297022989272},
                            'southwest': {'lat': -8.193948529892722,
                                          'lng': 15.37027057010728}}},
  'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png',
  'id': '083d7c8c7882ffdf1cbb65a43d3244773592c9cf',
  'name': 'Hotel Pif Paf',
  'photos': [{'height': 1080,
              'html_attributions': ['<a '
                                    'href="https://maps.google.com/maps/contrib/117139434344826469046">Staline '
                                    'Satola</a>'],
              'photo_reference': 'CmRaAAAA2NQRvQCU9EY1jqEUy-Pr-oiQAUCcV8q4mbQ35MskT44pfWci5bCneUYv5lURrsePil5fyA8bMEBTkd0O-MqDoZXjiYN6B0FGdf55TDhtVWmNqVD4oMFV7ZrEcpEMPM43EhBqFnDPHsvbB5zRmOZZqgARGhSIfAoPyqNwDfUoHrenH8vc_AeFRQ',
              'width': 1920}],
  'p

[{'geometry': {'location': {'lat': -20.3085941, 'lng': 30.0458761},
               'viewport': {'northeast': {'lat': -20.30710767010727,
                                          'lng': 30.04762827989272},
                            'southwest': {'lat': -20.30980732989272,
                                          'lng': 30.04492862010728}}},
  'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png',
  'id': 'fbb176f81b9d323c9bcd8b23acc69098b2179f00',
  'name': 'Pote Hill Hotel',
  'photos': [{'height': 1836,
              'html_attributions': ['<a '
                                    'href="https://maps.google.com/maps/contrib/104290751411623703402">Victor '
                                    'Mavingire</a>'],
              'photo_reference': 'CmRaAAAADOPrPwSYbSVsl0kJt51iup1HUSdXoyZBdho6pmZDV1fl8zwYgrxTV5h5zdQv1xJwcRt0dgq7ej8qQqSfEfu56nJ9d7zJUPVsK7FB9MnJ45ormiMok-67nY6b_gx6-RzSEhDGznCaT4aDx5edj7COZ-gqGhSYy8QxAIAk5d6LJwkp7Wwqmiotrg',
              'width': 3264}],

[{'geometry': {'location': {'lat': 38.013646, 'lng': -121.8699306},
               'viewport': {'northeast': {'lat': 38.01497792989272,
                                          'lng': -121.8685452701073},
                            'southwest': {'lat': 38.01227827010728,
                                          'lng': -121.8712449298927}}},
  'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png',
  'id': 'b450db092151dea59192908c7653b506c4491be6',
  'name': 'Hampton Inn & Suites Pittsburg',
  'opening_hours': {'open_now': True},
  'photos': [{'height': 500,
              'html_attributions': ['<a '
                                    'href="https://maps.google.com/maps/contrib/115025775633965596821">Hampton '
                                    'Inn &amp; Suites Pittsburg</a>'],
              'photo_reference': 'CmRaAAAAqznVsiNSd1nt2AQgRAN3BwVCAfXBuaPEPL56BZdQ9fAcMqVZhUCPHN5tjoPTyDNpCcJmSsCTvOAufqyCr1IcO4WNxBWC_hct0hKde4rF90dPFLAZmg_zETMjYfRvTfh4EhCB488Tld6jTKgk

[{'geometry': {'location': {'lat': -0.817912, 'lng': 29.78866},
               'viewport': {'northeast': {'lat': -0.8165621701072777,
                                          'lng': 29.79000982989272},
                            'southwest': {'lat': -0.8192618298927222,
                                          'lng': 29.78731017010728}}},
  'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png',
  'id': '0e1f0e2dd6c28a9675cbb4573e6182952b070037',
  'name': 'Kigezi Forest Cottages',
  'place_id': 'ChIJ3VzA3n-V3hkRBkExX1-rRig',
  'plus_code': {'compound_code': '5QJQ+RF Kambuga, Uganda',
                'global_code': '6GFF5QJQ+RF'},
  'rating': 0,
  'reference': 'ChIJ3VzA3n-V3hkRBkExX1-rRig',
  'scope': 'GOOGLE',
  'types': ['lodging', 'point_of_interest', 'establishment'],
  'user_ratings_total': 0,
  'vicinity': '48, Kambuga'},
 {'geometry': {'location': {'lat': -0.8065030999999999, 'lng': 29.7107325},
               'viewport': {'northeast': {'lat': -0.805153270

In [25]:
ideal_vacation_location = ideal_vacation_spot.rename(columns={"city_ID":"City ID", "city": "City", "cloudiness":"Cloudiness", "country":"Country", "date":"Date", "humidity":"Humidity", "latitude":"Latitude", "longitude":"Longitude", "max_temp":"Max Temp", "wind_speed":"Wind Speed", "hotel name":"Hotel Name"})
ideal_vacation_location


,City ID,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temp,Wind Speed,Hotel Name,hotel,hotel rating
25,25,camabatela,100.0,AO,04/12/2020,70.0,-8.19,15.38,65.12,3.04,Hotel Pif Paf,3.0,3.0
238,238,lyantonde,100.0,UG,04/12/2020,63.0,-0.40,31.16,65.35,3.11,Courtyard International Hotel Uganda,4.0,4.0
284,284,kibondo,100.0,TZ,04/12/2020,66.0,-3.59,30.72,62.46,1.88,Swidish Modern Hotel,3.8,3.8
308,308,zvishavane,77.0,ZW,04/12/2020,35.0,-20.33,30.07,67.37,2.66,Pote Hill Hotel,4.0,4.0
433,433,along,97.0,IN,04/12/2020,54.0,28.17,94.77,63.97,4.14,Hotel Toshi Palace : Dine Dwell Drink,3.5,3.5
484,484,pittsburg,90.0,US,04/12/2020,67.0,38.03,-121.88,64.99,3.80,Hampton Inn & Suites Pittsburg,4.3,4.3
544,544,katete,100.0,UG,04/12/2020,61.0,-0.78,29.75,65.39,3.31,Kigezi Forest Cottages,0.0,0.0
548,548,zambezi,60.0,ZM,04/12/2020,62.0,-13.54,23.10,66.27,3.89,Royal Kutachika Lodge,3.9,3.9
579,579,chiradzulu,30.0,MW,04/12/2020,70.0,-15.67,35.14,62.82,2.95,Zomba Road Country Lodge,NaN,5.0


In [26]:
# NOTE: Do not change any of the code in this cell
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Rating</dt><dd>{hotel rating}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in ideal_vacation_location.iterrows()]
locations = ideal_vacation_location[["Latitude", "Longitude"]]

In [27]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

# Display Map
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='500px', width='650px'))